# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
clicks = pd.read_pickle("../../../../data/tp2/clicks_tp2_formateado.pkl")

---

# Armado de ventanas

In [4]:
for ventana_nro in range(1,6):
    clicks["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= clicks["created"].dt.day) & ((clicks["created"].dt.day) <= (19+ventana_nro)))

---

# Creación de features

## Creación de sets de entrenamiento

In [23]:
Xs = {}
for ventana_nro in range(1,6):
    Xs[ventana_nro] = clicks.loc[clicks["ventana_{}".format(ventana_nro)]]["ref_hash"].drop_duplicates().to_frame().set_index("ref_hash")

---

## Función generadora de features

In [6]:
def generar_feature_en_ventanas(dataframe, generador_feature, destinos):
    for ventana_nro in range(1, 6):
        feature = generador_feature(dataframe.loc[dataframe["ventana_{}".format(ventana_nro)]], ventana_nro)
        destinos[ventana_nro] = destinos[ventana_nro].merge(feature, left_index=True, right_index=True)

---

## Estadísticas sobre el tiempo hasta clickear

In [24]:
def tiempo_hasta_clickear_media(dataframe, nro_ventana):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "mean"}).rename(columns={"timeToClick": "tiempo_hasta_click_mean"})

In [25]:
generar_feature_en_ventanas(clicks, tiempo_hasta_clickear_media, Xs)

In [26]:
def tiempo_hasta_clickear_max(dataframe, nro_ventana):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "max"}).rename(columns={"timeToClick": "tiempo_hasta_click_max"})

In [27]:
generar_feature_en_ventanas(clicks, tiempo_hasta_clickear_max, Xs)

In [28]:
def tiempo_hasta_clickear_min(dataframe, nro_ventana):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "min"}).rename(columns={"timeToClick": "tiempo_hasta_click_min"})

In [29]:
generar_feature_en_ventanas(clicks, tiempo_hasta_clickear_min, Xs)

In [30]:
def tiempo_hasta_clickear_std(dataframe, nro_ventana):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "std"}).rename(columns={"timeToClick": "tiempo_hasta_click_std"}).fillna(0)

In [31]:
generar_feature_en_ventanas(clicks, tiempo_hasta_clickear_std, Xs)

---

## Cantidad de clicks dentro de la ventana

In [34]:
def cantidad_de_clicks(dataframe, nro_ventana):
    return dataframe.groupby("ref_hash").agg({"timeToClick" : "count"}).rename(columns={"timeToClick": "cantidad_clicks"})

In [35]:
generar_feature_en_ventanas(clicks, cantidad_de_clicks, Xs)

---

## Cantidad de clicks discriminando por períodos

In [36]:
# Creo columnas para luego separar
clicks['hora'] = clicks['created'].dt.hour
clicks['dia'] = clicks['created'].dt.day
clicks['dia_de_la_semana'] = clicks['created'].dt.dayofweek

In [40]:
def cantidad_apariciones_por_hora(dataframe, nro_ventana):
    apariciones_por_hora = dataframe.groupby(["ref_hash", "hora"]).agg(({"created" : "count"}))
    apariciones_por_hora = apariciones_por_hora.unstack().fillna(0).astype(np.int64)
    apariciones_por_hora.columns = ["apariciones_en_hora_{}".format(hora) for hora in range(0,24)]
    return apariciones_por_hora

In [41]:
generar_feature_en_ventanas(clicks, cantidad_apariciones_por_hora, Xs)

In [42]:
def cantidad_apariciones_por_dia(dataframe, nro_ventana):
    dias = dataframe["dia"].drop_duplicates
    apariciones_por_dia = dataframe.groupby(["ref_hash", "dia"]).agg(({"created" : "count"}))
    apariciones_por_dia = apariciones_por_dia.unstack().fillna(0).astype(np.int64)
    apariciones_por_dia.columns = ["apariciones_en_dia_{}".format(dia) for dia in range(1,4)]
    return apariciones_por_dia

In [43]:
generar_feature_en_ventanas(clicks, cantidad_apariciones_por_dia, Xs)

---

## Tiempo medio entre clicks

In [45]:
def tiempo_medio_entre_clicks(dataframe, nro_ventana):
    gb = dataframe.groupby("ref_hash")
    ps = {}
    for t, g in gb:
        ps[t] = g["created"].sub(g["created"].shift()).mean()
    return pd.Series(ps).dt.total_seconds().fillna(0).to_frame().rename(columns={0: "tiempo_medio_entre_clicks"})

In [46]:
generar_feature_en_ventanas(clicks, tiempo_medio_entre_clicks, Xs)

---

# Guardado de los sets de entrenamiento

In [48]:
for ventana_nro in range(1, 6):
    Xs[ventana_nro].to_pickle("../../features/clicks_train_ventana_{}.pkl".format(ventana_nro))
    Xs[ventana_nro].to_csv("../../features/clicks_train_ventana_{}.csv".format(ventana_nro))

---